<a href="https://colab.research.google.com/github/Daniel022de/Bootcamp_SoulCode_EngenhariaDados/blob/main/Google%20Cloud/conex%C3%A3o_GoogleCloudStorage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SPARK - CONEXÃO DO COLAB AO GOOGLE CLOUD STORAGE**

Esse Notebook é referente as instruções feitas pelo professor **Bismark William** no bootcamp **engenharia de dados** da **SoulCode** visando a consolidação dos conhecimentos sobre a conexão do colab a um bucket privado da **Google Cloud Storage** e a utilidade do Pyspark para criar um DataFrame originado desse processo. O principal propósito desse tipo de conexão é garantir a segurança na extração do Dataset.







! **Você pode encontrar esse notebook no meu repositório** [GitHub](https://github.com/Daniel022de/Bootcamp_SoulCode_EngenhariaDados)

! **Você pode entrar em contato comigo através do meu email** ddololiveira.pessoal@gmail.com **e** [Linkedin](https://www.linkedin.com/in/daniel-oliveira-503b0323b/).

! **Toda dúvida,recomendações e feedbacks serão bem-vindas.**



# Instalação

In [20]:
# Pyspark e Google Cloud Storage File System
!pip install pyspark
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importação

In [21]:
from pyspark.sql import SparkSession
from google.cloud import storage
import pandas as pd
import os

# Chave de segurança

In [22]:
ServiceAccount = '/content/drive/MyDrive/Colab Notebooks/bc26-aulas-7791d7a8a1e0.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ServiceAccount

#Setando a Bucket

In [23]:
storage.Client().get_bucket('ed7-datasets').blob('spotify.csv')
path = 'gs://ed7-datasets/spotify.csv'

# Configurando variável de ambiente (spark)

In [24]:
spark =  (
    SparkSession.builder
                .master('local')
                .appName('gcsfs2')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

# Lendo o path 

## Pandas

In [25]:
import pandas as pd
df_pandas = pd.read_csv(path,sep=',')

In [26]:
df_pandas

,Unnamed: 0,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,0,Boulevard of Broken Dreams,73,262333,0.005520000000000001kg,0.496mol/L,0.682,2.94e-05,8.0,0.0589,-4.095,1,0.0294,167.06,4,0.474
1,1,In The End,66,216933,0.0103kg,0.542mol/L,0.853,0.0,3.0,0.108,-6.407,0,0.0498,105.256,4,0.370
2,2,Seven Nation Army,76,231733,0.00817kg,0.737mol/L,0.463,0.447,0.0,0.255,-7.8279999999999985,1,0.0792,123.881,4,0.324
3,3,By The Way,74,216933,0.0264kg,0.451mol/L,0.97,0.00355,0.0,0.102,-4.938,1,0.107,122.444,4,0.198
4,4,How You Remind Me,56,223826,0.000954kg,0.447mol/L,0.7659999999999999,0.0,10.0,0.113,-5.065,1,0.0313,172.011,4,0.574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18830,18830,Let It Breathe,60,159645,0.893kg,0.5mol/L,0.151,6.54e-05,11.0,0.111,-16.107,1,0.0348,113.969,4,0.300
18831,18831,Answers,60,205666,0.765kg,0.495mol/L,0.161,1.33e-06,11.0,0.105,-14.078,0,0.0301,94.286,4,0.265
18832,18832,Sudden Love (Acoustic),23,182211,0.847kg,0.7190000000000001mol/L,0.325,0.0,0.0,0.125,-12.222,1,0.0355,130.534,4,0.286
18833,18833,Gentle on My Mind,55,352280,0.945kg,0.488mol/L,0.326,0.0157,3.0,0.119,-12.02,1,0.0328,106.063,4,0.323


## Pyspark

In [27]:
df_pyspark = spark.read.format('csv').option('inferschema','true').option('header','true').load(path,sep=',')

In [28]:
df_pyspark.show()

+---+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+----+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|_c0|           song_name|song_popularity|song_duration_ms|        acousticness|        danceability|            energy|    instrumentalness| key|           liveness|           loudness|audio_mode|         speechiness|  tempo|time_signature|     audio_valence|
+---+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+----+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|  0|Boulevard of Brok...|             73|          262333|0.005520000000000...|          0.496mol/L|             0.682|            2.94e-05| 8.0|             0.0589|             -4.095|         1|              0.0294